In [18]:
# imports 
import os
import pandas as pd
from PIL import Image
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

In [19]:
# variables 
train_csv = "train.csv" # replace with your own train label file path
val_csv   = "val.csv" # replace with your own validation label file path
test_csv  = "offsite_test.csv"  # replace with your own test label file path
train_image_dir ="./images/train"   # replace with your own train image floder path
val_image_dir = "./images/val"  # replace with your own validation image floder path
test_image_dir = "./images/offsite_test" # replace with your own test image floder path
#pretrained_backbone = './pretrained_backbone/ckpt_resnet18_ep50.pt'  # replace with your own pretrained backbone path
pretrained_backbone = './pretrained_backbone/ckpt_efficientnet_ep50.pt'
#backbone = 'resnet18'  # backbone choices: ["resnet18", "efficientnet"]
backbone = 'efficientnet'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset preparation methods from template 

In [20]:
# ========================
# Dataset preparation
# ========================
class RetinaMultiLabelDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            row = self.data.iloc[idx]
            img_path = os.path.join(self.image_dir, row.iloc[0])
    
            # check if file exists
            if not os.path.exists(img_path):
                raise FileNotFoundError(f"Image not found: {img_path}")
                
            img = Image.open(img_path).convert("RGB")
            labels = torch.tensor(row[1:].values.astype("float32"))
            if self.transform:
                img = self.transform(img)
            return img, labels
        except Exception as e:
            print(f"Error loading image at index {idx}: {str(e)}")

In [21]:
# ========================
# build model
# ========================
def build_model(backbone="resnet18", num_classes=3, pretrained=True):

    if(pretrained):
        weights_resnet = models.ResNet18_Weights.DEFAULT
        weights_efficient_net = models.EfficientNet_B0_Weights.DEFAULT
    else:
        weights_resnet = weights_efficient_net = None

    if backbone == "resnet18":
        model = models.resnet18(weights_resnet)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif backbone == "efficientnet":
        model = models.efficientnet_b0(weights_efficient_net)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Unsupported backbone")
    return model


## Implementing Patience feature so we don't need to run all the epochs if the model no longer improves
Early stopping monitors a validation metric (usually loss). If it doesn’t improve for N consecutive epochs, training stops automatically.

This prevents overfitting and saves compute — especially useful when you're iterating on models in your scientific workflows.

In [22]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.should_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            return

        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True

In [23]:
# ========================
# model training and val
# ========================
#def train_model(backbone, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, 
#                       epochs=10, batch_size=32, lr=1e-4, img_size=256, save_dir="checkpoints",pretrained_backbone=None):
def train_model(model, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, optimizer, criterion=None, scheduler=None, epochs=10, model_type="frozen-backbone",
               cb_loss=None):
    since = time.time()

    img_size = 256 
    batch_size = 32
    #lr = 1e-4
    save_dir = "checkpoints"

    # transforms: data augmentation and normalization for training
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]), # imagenet normalization stats
    ])

    # datasets & dataloaders
    ds_paths = {'train': [train_csv, train_image_dir], 'val': [val_csv, val_image_dir]}
    image_datasets = {x: RetinaMultiLabelDataset(ds_paths[x][0], ds_paths[x][1], transform)
                      for x in ['train', 'val']}

    dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=(x =='train'), num_workers=0)
                      for x in ['train', 'val']}
    
    # loss function
    if criterion is None:
        criterion = nn.BCEWithLogitsLoss()

    # training
    best_val_loss = float("inf")
    os.makedirs(save_dir, exist_ok=True)
    ckpt_path = os.path.join(save_dir, f"best_{backbone}_{model_type}.pt")

    # early stop mechanism 
    early_stopper = EarlyStopping(patience=5, min_delta=0.001)
    
    for epoch in range(epochs):
        print(f'Epoch {epoch}/{epochs - 1}')
        print('-' * 10)

        # each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # set model to training mode
            else:
                model.eval() # set model to evaluate mode

            train_loss = 0
            val_loss = 0

            # iterate over data
            #for imgs, labels in train_loader:
            for imgs, labels in dataloaders[phase]:
                imgs, labels = imgs.to(device), labels.to(device) # copy data and labels to gpu
                optimizer.zero_grad() # zero the parameters gradients to avoid accumulating 

                # forward step
                # track history only if in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(imgs)

                    if cb_loss is not None:
                        loss = cb_loss.compute_CB_loss(outputs, labels)
                    else:
                        loss = criterion(outputs, labels)

                    # backward step + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        train_loss += loss.item() * imgs.size(0)
                    else: 
                        val_loss += loss.item() * imgs.size(0)
                        
            if phase == 'train':
                train_loss /= len(dataloaders['train'].dataset)
                print(f"[{backbone}] {phase}: Epoch {epoch+1}/{epochs} Train Loss: {train_loss:.4f}")
                if scheduler is not None:
                    scheduler.step()
            else:
                val_loss /= len(dataloaders['val'].dataset)
                print(f"[{backbone}] {phase}: Epoch {epoch+1}/{epochs} Val Loss: {val_loss:.4f}")
            
                # save best
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    torch.save(model.state_dict(), ckpt_path)
                    print(f"Saved best model for {backbone} at {ckpt_path} with val loss: {best_val_loss}")

                early_stopper(val_loss)
                
        if early_stopper.should_stop:
            print("Patience has run out! Stopping!")
            break
    print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best eval loss: {best_val_loss:4f}')

        # load best model weights
    model.load_state_dict(torch.load(ckpt_path, weights_only=True))
    return model

# Test Model Function 
#### I've copied the eval part of the training into a testing function in order to run 
#### inference on the test set with the untouched backbones as part of the first task  

In [24]:
def test_model(model, test_dataset, pretrained=True, img_size=256,  batch_size=32, num_workers=0):
    # ========================
    # testing
    # ========================

    # transforms
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ])
    
    test_ds  = RetinaMultiLabelDataset(test_dataset, test_image_dir, transform)
    test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > 0.5).astype(int)
            y_true.extend(labels.numpy())
            y_pred.extend(preds)

    y_true = torch.tensor(y_true).numpy()
    y_pred = torch.tensor(y_pred).numpy()

    disease_names = ["DR", "Glaucoma", "AMD"]

    avg_f_score = 0.0
    for i, disease in enumerate(disease_names):  #compute metrics for every disease
        y_t = y_true[:, i]
        y_p = y_pred[:, i]

        acc = accuracy_score(y_t, y_p)
        precision = precision_score(y_t, y_p, average="macro",zero_division=0)
        recall = recall_score(y_t, y_p, average="macro",zero_division=0)
        f1 = f1_score(y_t, y_p, average="macro",zero_division=0)
        kappa = cohen_kappa_score(y_t, y_p)
        avg_f_score += f1

        print(f"{disease} Results [{backbone}]")
        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall   : {recall:.4f}")
        print(f"F1-score : {f1:.4f}")
        print(f"Kappa    : {kappa:.4f}")
    
    avg_f_score /= 3
    print("-" * 10)
    print(f"Average F1-score : {f1:.4f}")
    

# TASK 1.1: NO FINE-TUNING
## In this task we evaluate both backbones directly on the ODIR test set without any manipulation

### Test-set with Resnet18

In [35]:
pretrained_backbone = './pretrained_backbone/ckpt_resnet18_ep50.pt'
backbone = 'resnet18'

model = build_model('resnet18', num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model.load_state_dict(state_dict)

test_model(model, test_csv)

C:\Users\netos\miniconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


DR Results [resnet18]
Accuracy : 0.5150
Precision: 0.5170
Recall   : 0.5202
F1-score : 0.4958
Kappa    : 0.0339
Glaucoma Results [resnet18]
Accuracy : 0.7850
Precision: 0.7063
Recall   : 0.6784
F1-score : 0.6893
Kappa    : 0.3804
AMD Results [resnet18]
Accuracy : 0.7850
Precision: 0.6305
Recall   : 0.7597
F1-score : 0.6472
Kappa    : 0.3211


### Test-set with efficientNet

In [36]:
pretrained_backbone = './pretrained_backbone/ckpt_efficientnet_ep50.pt'
backbone = 'efficientnet'

model = build_model('efficientnet', num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model.load_state_dict(state_dict)

test_model(model, test_csv)

C:\Users\netos\miniconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


DR Results [efficientnet]
Accuracy : 0.6000
Precision: 0.5588
Recall   : 0.5667
F1-score : 0.5575
Kappa    : 0.1228
Glaucoma Results [efficientnet]
Accuracy : 0.7950
Precision: 0.7243
Recall   : 0.7333
F1-score : 0.7285
Kappa    : 0.4571
AMD Results [efficientnet]
Accuracy : 0.7150
Precision: 0.6041
Recall   : 0.7403
F1-score : 0.5946
Kappa    : 0.2482


# Task 1.2 Frozen backbone, fine-tunning classifier only 
#### In this task we'll do what's called "Feature Extraction", we'll freeze all the network, except the final layer. 
#### We need to set `requires_grad = False` to freeze the parameters so that the gradients are not computer in `backward()`

## Feature Extraction with Resnet18

In [71]:
pretrained_backbone = './pretrained_backbone/ckpt_resnet18_ep50.pt'
backbone = 'resnet18'

model_conv = build_model(backbone, num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model_conv.load_state_dict(state_dict)

for param in model_conv.parameters():
    param.requires_grad = False

# rebuilding classifier layer
# parameters of newly contructed modules have requires_grad=True by default
num_classes = 3
model_conv.fc = nn.Linear(model_conv.fc.in_features, num_classes)

# Only parameters of final layer are being optimized as
optimizer = optim.Adam(model_conv.fc.parameters(), lr=1e-4)

C:\Users\netos\miniconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


### Train and Evaluate

In [72]:
model_conv = train_model(model_conv, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, optimizer=optimizer, epochs=25)

Epoch 0/24
----------
[resnet18] train: Epoch 1/25 Train Loss: 0.6760
[resnet18] val: Epoch 1/25 Val Loss: 0.6420
Saved best model for resnet18 at checkpoints\best_resnet18.pt with val loss: 0.6420445466041564
Epoch 1/24
----------
[resnet18] train: Epoch 2/25 Train Loss: 0.5592
[resnet18] val: Epoch 2/25 Val Loss: 0.6078
Saved best model for resnet18 at checkpoints\best_resnet18.pt with val loss: 0.6078139472007752
Epoch 2/24
----------
[resnet18] train: Epoch 3/25 Train Loss: 0.5262
[resnet18] val: Epoch 3/25 Val Loss: 0.5920
Saved best model for resnet18 at checkpoints\best_resnet18.pt with val loss: 0.5919931149482727
Epoch 3/24
----------
[resnet18] train: Epoch 4/25 Train Loss: 0.5131
[resnet18] val: Epoch 4/25 Val Loss: 0.5829
Saved best model for resnet18 at checkpoints\best_resnet18.pt with val loss: 0.5828698968887329
Epoch 4/24
----------
[resnet18] train: Epoch 5/25 Train Loss: 0.5023
[resnet18] val: Epoch 5/25 Val Loss: 0.5734
Saved best model for resnet18 at checkpoints\b

In [73]:
test_model(model_conv, test_csv)

DR Results [resnet18]
Accuracy : 0.7500
Precision: 0.7091
Recall   : 0.6405
F1-score : 0.6523
Kappa    : 0.3207
Glaucoma Results [resnet18]
Accuracy : 0.7700
Precision: 0.7208
Recall   : 0.5513
F1-score : 0.5362
Kappa    : 0.1416
AMD Results [resnet18]
Accuracy : 0.9050
Precision: 0.8131
Recall   : 0.6080
F1-score : 0.6468
Kappa    : 0.3081


## Feature Extraction with EfficientNet

In [74]:
pretrained_backbone = './pretrained_backbone/ckpt_efficientnet_ep50.pt'
backbone = 'efficientnet'

model_conv_effnet = build_model(backbone, num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model_conv_effnet.load_state_dict(state_dict)

for param in model_conv_effnet.parameters():
    param.requires_grad = False

# rebuilding classifier layer
# parameters of newly contructed modules have requires_grad=True by default
num_classes = 3
model_conv_effnet.classifier[1] = nn.Linear(model_conv_effnet.classifier[1].in_features, num_classes)

# Only parameters of final layer are being optimized
optimizer = optim.Adam(model_conv_effnet.classifier[1].parameters(), lr=1e-4)

C:\Users\netos\miniconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


### Train and Evaluate

In [75]:
model_conv_effnet = train_model(model_conv_effnet, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, optimizer=optimizer, epochs=25)

Epoch 0/24
----------
[efficientnet] train: Epoch 1/25 Train Loss: 0.6259
[efficientnet] val: Epoch 1/25 Val Loss: 0.5931
Saved best model for efficientnet at checkpoints\best_efficientnet.pt with val loss: 0.5930671834945679
Epoch 1/24
----------
[efficientnet] train: Epoch 2/25 Train Loss: 0.5371
[efficientnet] val: Epoch 2/25 Val Loss: 0.5663
Saved best model for efficientnet at checkpoints\best_efficientnet.pt with val loss: 0.5662716507911683
Epoch 2/24
----------
[efficientnet] train: Epoch 3/25 Train Loss: 0.5024
[efficientnet] val: Epoch 3/25 Val Loss: 0.5488
Saved best model for efficientnet at checkpoints\best_efficientnet.pt with val loss: 0.5488045358657837
Epoch 3/24
----------
[efficientnet] train: Epoch 4/25 Train Loss: 0.4907
[efficientnet] val: Epoch 4/25 Val Loss: 0.5392
Saved best model for efficientnet at checkpoints\best_efficientnet.pt with val loss: 0.5391703486442566
Epoch 4/24
----------
[efficientnet] train: Epoch 5/25 Train Loss: 0.4670
[efficientnet] val: Ep

In [76]:
test_model(model_conv_effnet, test_csv)

DR Results [efficientnet]
Accuracy : 0.7750
Precision: 0.7385
Recall   : 0.6917
F1-score : 0.7058
Kappa    : 0.4171
Glaucoma Results [efficientnet]
Accuracy : 0.8100
Precision: 0.7600
Recall   : 0.6743
F1-score : 0.6974
Kappa    : 0.4043
AMD Results [efficientnet]
Accuracy : 0.9250
Precision: 0.8747
Recall   : 0.6989
F1-score : 0.7523
Kappa    : 0.5095


# Task 1.3 Full Fine-tuning: Both backbone and classifier are updated on ODIR training set 
#### We'll load the pretrained models, resetting the final fully connected layer
#### as the network is trained, the weights are modified during backpropagation

## Fine-tuning Resnet18

In [9]:
pretrained_backbone = './pretrained_backbone/ckpt_resnet18_ep50.pt'
backbone = 'resnet18'

model_ft_resnet = build_model(backbone, num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model_ft_resnet.load_state_dict(state_dict)

for p in model_ft_resnet.parameters():
        p.requires_grad = True

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_ft_resnet.parameters()), lr=1e-4)

## Train and Evaluate

In [11]:
model_ft_resnet = train_model(model_ft_resnet, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, optimizer=optimizer, epochs=25, model_type="fine_tune")

Epoch 0/24
----------
[resnet18] train: Epoch 1/25 Train Loss: 0.8100
[resnet18] val: Epoch 1/25 Val Loss: 0.6989
Saved best model for resnet18 at checkpoints\best_resnet18_fine_tune.pt with val loss: 0.6988784217834473
Epoch 1/24
----------
[resnet18] train: Epoch 2/25 Train Loss: 0.2874
[resnet18] val: Epoch 2/25 Val Loss: 0.5162
Saved best model for resnet18 at checkpoints\best_resnet18_fine_tune.pt with val loss: 0.5162098836898804
Epoch 2/24
----------
[resnet18] train: Epoch 3/25 Train Loss: 0.1737
[resnet18] val: Epoch 3/25 Val Loss: 0.4896
Saved best model for resnet18 at checkpoints\best_resnet18_fine_tune.pt with val loss: 0.48957729935646055
Epoch 3/24
----------
[resnet18] train: Epoch 4/25 Train Loss: 0.0805
[resnet18] val: Epoch 4/25 Val Loss: 0.5275
Epoch 4/24
----------
[resnet18] train: Epoch 5/25 Train Loss: 0.0455
[resnet18] val: Epoch 5/25 Val Loss: 0.5222
Epoch 5/24
----------
[resnet18] train: Epoch 6/25 Train Loss: 0.0285
[resnet18] val: Epoch 6/25 Val Loss: 0.52

In [12]:
test_model(model_ft_resnet, test_csv)

DR Results [resnet18]
Accuracy : 0.8150
Precision: 0.7809
Recall   : 0.7726
F1-score : 0.7765
Kappa    : 0.5531
Glaucoma Results [resnet18]
Accuracy : 0.8900
Precision: 0.8803
Recall   : 0.8100
F1-score : 0.8371
Kappa    : 0.6759
AMD Results [resnet18]
Accuracy : 0.9350
Precision: 0.8730
Recall   : 0.7643
F1-score : 0.8064
Kappa    : 0.6142
----------
Average F1-score : 0.8064


C:\Users\netos\AppData\Local\Temp\ipykernel_22256\4137734398.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  y_true = torch.tensor(y_true).numpy()


## Fine Tuning EfficientNet

In [35]:
pretrained_backbone = './pretrained_backbone/ckpt_efficientnet_ep50.pt'
backbone = 'efficientnet'

model_ft_effnet = build_model(backbone, num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model_ft_effnet.load_state_dict(state_dict)

for p in model_ft_effnet.parameters():
        p.requires_grad = True

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_ft_effnet.parameters()), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs (model was not anymore learning after 5 epoch on full fine tuning)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

C:\Users\netos\miniconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


### Train and evaluate

In [36]:
model_ft_effnet = train_model(model_ft_effnet, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, scheduler=scheduler,
                                optimizer=optimizer, epochs=25, model_type="fine_tune")

Epoch 0/24
----------
[efficientnet] train: Epoch 1/25 Train Loss: 0.5473
[efficientnet] val: Epoch 1/25 Val Loss: 0.4415
Saved best model for efficientnet at checkpoints\best_efficientnet_fine_tune.pt with val loss: 0.44152442216873167
Epoch 1/24
----------
[efficientnet] train: Epoch 2/25 Train Loss: 0.2281
[efficientnet] val: Epoch 2/25 Val Loss: 0.7603
Epoch 2/24
----------
[efficientnet] train: Epoch 3/25 Train Loss: 0.1249
[efficientnet] val: Epoch 3/25 Val Loss: 0.5108
Epoch 3/24
----------
[efficientnet] train: Epoch 4/25 Train Loss: 0.0950
[efficientnet] val: Epoch 4/25 Val Loss: 0.6312
Epoch 4/24
----------
[efficientnet] train: Epoch 5/25 Train Loss: 0.1030
[efficientnet] val: Epoch 5/25 Val Loss: 0.7067
Epoch 5/24
----------
[efficientnet] train: Epoch 6/25 Train Loss: 0.0984
[efficientnet] val: Epoch 6/25 Val Loss: 0.6420
Epoch 6/24
----------
[efficientnet] train: Epoch 7/25 Train Loss: 0.0838
[efficientnet] val: Epoch 7/25 Val Loss: 0.5184
Epoch 7/24
----------
[efficien

In [37]:
test_model(model_ft_effnet, test_csv)

DR Results [efficientnet]
Accuracy : 0.7600
Precision: 0.7270
Recall   : 0.7571
F1-score : 0.7345
Kappa    : 0.4737
Glaucoma Results [efficientnet]
Accuracy : 0.8700
Precision: 0.8704
Recall   : 0.7623
F1-score : 0.7969
Kappa    : 0.5988
AMD Results [efficientnet]
Accuracy : 0.8600
Precision: 0.6763
Recall   : 0.7421
F1-score : 0.7003
Kappa    : 0.4037
----------
Average F1-score : 0.7003


# Task 2.1: Implementing a Focal Loss function
## Focal Loss is a powerful technical for dealing with class imbalance by focusing training on hard, misclassified examples.

##### Focal Loss adds a modulating factor to cross‑entropy so that easy 
##### examples are down‑weighted and hard examples get more focus. 
##### This is widely used in object detection models like RetinaNet

## Implementing Focal Loss in pytorch

In [21]:
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25, reduction='mean'):
        super(FocalLoss, self).__init__()
        """
        :param gamma: Focusing parameter, controls the strength of the modulating factor (1 - p_t)^gamma
        :param alpha: Balancing factor, can be a scalar or a tensor for class-wise weights. If None, no class balancing is used.
        :param reduction: Specifies the reduction method: 'none' | 'mean' | 'sum'
        """
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        """ Focal Loss for Multi-label classification """
        """
        inputs: tensor of shape (batch, num_classes)
        targets: tensor of shape (batch, num_classes) with 0/1 labels
        """
        probs = torch.sigmoid(inputs)

        # Compute binary cross entropy
        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')

        # compute focal weight
        pt = probs * targets + (1 - probs) * (1 - targets)
        focal_weight = (1 - pt) ** self.gamma

        # apply alpha if provided
        if self.alpha is not None:
            alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
            bce_loss = alpha_t * bce_loss

        # apply focal loss weight
        loss = focal_weight * bce_loss
 
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

## Fine tuning resnet18 with Focal Loss 

In [22]:
pretrained_backbone = './pretrained_backbone/ckpt_resnet18_ep50.pt'
backbone = 'resnet18'

model_fc_ft_resnet = build_model(backbone, num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model_fc_ft_resnet.load_state_dict(state_dict)
for p in model_fc_ft_resnet.parameters():
        p.requires_grad = True
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_fc_ft_resnet.parameters()), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Loss function using Focal Loss method
criterion = FocalLoss(alpha=None) 

C:\Users\netos\miniconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [23]:
model_ft_effnet = train_model(model_fc_ft_resnet, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, scheduler=scheduler,
                                criterion=criterion, optimizer=optimizer, epochs=25, model_type="focal_loss_fine_tuned")

Epoch 0/24
----------
[resnet18] train: Epoch 1/25 Train Loss: 0.6603
[resnet18] val: Epoch 1/25 Val Loss: 0.5030
Saved best model for resnet18 at checkpoints\best_resnet18_focal_loss_fine_tuned.pt with val loss: 0.5030284738540649
Epoch 1/24
----------
[resnet18] train: Epoch 2/25 Train Loss: 0.1288
[resnet18] val: Epoch 2/25 Val Loss: 0.2917
Saved best model for resnet18 at checkpoints\best_resnet18_focal_loss_fine_tuned.pt with val loss: 0.2916832408308983
Epoch 2/24
----------
[resnet18] train: Epoch 3/25 Train Loss: 0.0683
[resnet18] val: Epoch 3/25 Val Loss: 0.2621
Saved best model for resnet18 at checkpoints\best_resnet18_focal_loss_fine_tuned.pt with val loss: 0.26209314823150637
Epoch 3/24
----------
[resnet18] train: Epoch 4/25 Train Loss: 0.0467
[resnet18] val: Epoch 4/25 Val Loss: 0.2265
Saved best model for resnet18 at checkpoints\best_resnet18_focal_loss_fine_tuned.pt with val loss: 0.22646843910217285
Epoch 4/24
----------
[resnet18] train: Epoch 5/25 Train Loss: 0.0278


## Testing trained model

In [24]:
test_model(model_fc_ft_resnet, test_csv)

DR Results [resnet18]
Accuracy : 0.7900
Precision: 0.7510
Recall   : 0.7643
F1-score : 0.7567
Kappa    : 0.5139
Glaucoma Results [resnet18]
Accuracy : 0.8850
Precision: 0.8686
Recall   : 0.8067
F1-score : 0.8311
Kappa    : 0.6636
AMD Results [resnet18]
Accuracy : 0.9050
Precision: 0.7578
Recall   : 0.7474
F1-score : 0.7525
Kappa    : 0.5050
----------
Average F1-score : 0.7525


C:\Users\netos\AppData\Local\Temp\ipykernel_15480\4137734398.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  y_true = torch.tensor(y_true).numpy()


## Fine tuning EfficientNet with Focal Loss 

In [26]:
pretrained_backbone = './pretrained_backbone/ckpt_efficientnet_ep50.pt'
backbone = 'efficientnet'

model_fc_ft_effnet = build_model(backbone, num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model_fc_ft_effnet.load_state_dict(state_dict)
for p in model_fc_ft_effnet.parameters():
        p.requires_grad = True
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_fc_ft_effnet.parameters()), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Loss function using Focal Loss method
criterion = FocalLoss(alpha=None) 

In [27]:
model_fc_ft_effnet = train_model(model_fc_ft_effnet, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, scheduler=scheduler,
                                criterion=criterion, optimizer=optimizer, epochs=25, model_type="focal_loss_fine_tuned")

Epoch 0/24
----------
[efficientnet] train: Epoch 1/25 Train Loss: 0.8610
[efficientnet] val: Epoch 1/25 Val Loss: 0.5453
Saved best model for efficientnet at checkpoints\best_efficientnet_focal_loss_fine_tuned.pt with val loss: 0.5452643358707427
Epoch 1/24
----------
[efficientnet] train: Epoch 2/25 Train Loss: 0.2387
[efficientnet] val: Epoch 2/25 Val Loss: 0.3134
Saved best model for efficientnet at checkpoints\best_efficientnet_focal_loss_fine_tuned.pt with val loss: 0.31337508678436277
Epoch 2/24
----------
[efficientnet] train: Epoch 3/25 Train Loss: 0.1025
[efficientnet] val: Epoch 3/25 Val Loss: 0.2726
Saved best model for efficientnet at checkpoints\best_efficientnet_focal_loss_fine_tuned.pt with val loss: 0.272551486492157
Epoch 3/24
----------
[efficientnet] train: Epoch 4/25 Train Loss: 0.0722
[efficientnet] val: Epoch 4/25 Val Loss: 0.2394
Saved best model for efficientnet at checkpoints\best_efficientnet_focal_loss_fine_tuned.pt with val loss: 0.23936595916748046
Epoch 4

In [28]:
test_model(model_fc_ft_effnet, test_csv)

DR Results [efficientnet]
Accuracy : 0.7400
Precision: 0.7016
Recall   : 0.7238
F1-score : 0.7082
Kappa    : 0.4196
Glaucoma Results [efficientnet]
Accuracy : 0.8150
Precision: 0.7509
Recall   : 0.7327
F1-score : 0.7408
Kappa    : 0.4821
AMD Results [efficientnet]
Accuracy : 0.8550
Precision: 0.6758
Recall   : 0.7592
F1-score : 0.7033
Kappa    : 0.4118
----------
Average F1-score : 0.7033


# Task 2.2: Class-Balanced Loss
Class‑Balanced Loss (CB Loss) is a principled way to correct for class imbalance by re‑weighting each class according to its effective number of samples, rather than raw counts. It is widely used in deep learning tasks where minority classes would otherwise be ignored. 

## Implementing CB Loss in pytorch

In [25]:
import numpy as np 

class CB_Loss():
    def __init__(self, samples_per_cls, no_of_classes, beta, gamma):
        self.samples_per_cls = samples_per_cls
        self.no_of_classes = no_of_classes
        self.beta = beta
        self.gamma = gamma

    def compute_CB_loss(self, logits, labels):
        """Compute the Class Balanced Loss between `logits` and the ground truth `labels`.
    
        Class Balanced Loss: ((1-beta)/(1-beta^n))*Loss(labels, logits)
        where Loss is one of the standard losses used for Neural Networks.
    
        Args:
          labels: A int tensor of size [batch].  (targets)
          logits: A float tensor of size [batch, no_of_classes]. (inputs)
          samples_per_cls: A python list of size [no_of_classes]. Number of training samples belonging to each class.
          no_of_classes: total number of classes. int
          loss_type: string. One of "sigmoid", "focal", "softmax".
          beta: float. Hyperparameter for Class balanced loss.
          gamma: float. Hyperparameter for Focal loss.
    
        Returns:
          cb_loss: A float tensor representing class balanced loss
        """
        # Compute effective number for each class
        effective_num = 1.0 - torch.pow(self.beta, self.samples_per_cls)
        weights = (1.0 - self.beta) / effective_num
    
        # Normalize weights so they sum to num_classes
        weights = weights / torch.sum(weights) * self.no_of_classes
    
        #labels_one_hot = F.one_hot(labels, self.no_of_classes).float()
    
        #weights = torch.tensor(weights).float()
    
        # Reshape for broadcasting: (1, num_classes)
        weights = weights.unsqueeze(0)
        #weights = weights.repeat(labels_one_hot.shape[0],1) * labels_one_hot
        #weights = weights.sum(1)
        #weights = weights.unsqueeze(1)
        #weights = weights.repeat(1, self.no_of_classes)
    
        # cb loss for multi-label classficiation
        bce = F.binary_cross_entropy_with_logits(input = logits,target = labels, reduction='none')

        # apply class-balanced weights
        cb_loss = bce * weights
        
        return cb_loss.mean()

## Fine tuning resnet18 with CB Loss 

In [26]:
pretrained_backbone = './pretrained_backbone/ckpt_resnet18_ep50.pt'
backbone = 'resnet18'

model_cb_ft_resnet = build_model(backbone, num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model_cb_ft_resnet.load_state_dict(state_dict)
for p in model_cb_ft_resnet.parameters():
        p.requires_grad = True
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_cb_ft_resnet.parameters()), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# setting up of Class Balanced Loss method
no_classes = 3
samples_per_class = torch.tensor([517, 163, 142]) # DR, Glaucoma, AMD - as stated in the project pdf (which I'll just blindly trust)
CB_loss = CB_Loss(samples_per_cls=samples_per_class, no_of_classes=no_classes, beta=0.9999, gamma=2.0)

C:\Users\netos\miniconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


## Training and testing model

In [27]:
model_cb_ft_resnet = train_model(model_cb_ft_resnet, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, scheduler=scheduler,
                                criterion=None, optimizer=optimizer, epochs=25, model_type="class_balance_fine_tuned", cb_loss=CB_loss)

Epoch 0/24
----------
[resnet18] train: Epoch 1/25 Train Loss: 0.6130
[resnet18] val: Epoch 1/25 Val Loss: 0.5831
Saved best model for resnet18 at checkpoints\best_resnet18_class_balance_fine_tuned.pt with val loss: 0.5831085252761841
Epoch 1/24
----------
[resnet18] train: Epoch 2/25 Train Loss: 0.2520
[resnet18] val: Epoch 2/25 Val Loss: 0.4343
Saved best model for resnet18 at checkpoints\best_resnet18_class_balance_fine_tuned.pt with val loss: 0.4343432652950287
Epoch 2/24
----------
[resnet18] train: Epoch 3/25 Train Loss: 0.1322
[resnet18] val: Epoch 3/25 Val Loss: 0.4721
Epoch 3/24
----------
[resnet18] train: Epoch 4/25 Train Loss: 0.0707
[resnet18] val: Epoch 4/25 Val Loss: 0.4428
Epoch 4/24
----------
[resnet18] train: Epoch 5/25 Train Loss: 0.0291
[resnet18] val: Epoch 5/25 Val Loss: 0.4503
Epoch 5/24
----------
[resnet18] train: Epoch 6/25 Train Loss: 0.0197
[resnet18] val: Epoch 6/25 Val Loss: 0.4577
Epoch 6/24
----------
[resnet18] train: Epoch 7/25 Train Loss: 0.0138
[res

In [28]:
test_model(model_cb_ft_resnet, test_csv)

DR Results [resnet18]
Accuracy : 0.8100
Precision: 0.7735
Recall   : 0.7786
F1-score : 0.7759
Kappa    : 0.5519
Glaucoma Results [resnet18]
Accuracy : 0.8750
Precision: 0.8469
Recall   : 0.8000
F1-score : 0.8194
Kappa    : 0.6398
AMD Results [resnet18]
Accuracy : 0.9200
Precision: 0.8139
Recall   : 0.7360
F1-score : 0.7674
Kappa    : 0.5360
----------
Average F1-score : 0.7674


C:\Users\netos\AppData\Local\Temp\ipykernel_32288\4137734398.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  y_true = torch.tensor(y_true).numpy()


## Fine tuning efficientNet with CB Loss 

In [30]:
pretrained_backbone = './pretrained_backbone/ckpt_efficientnet_ep50.pt'
backbone = 'efficientnet'

model_cb_ft_effnet = build_model(backbone, num_classes=3, pretrained=False).to(device)
state_dict = torch.load(pretrained_backbone, map_location="cpu")
model_cb_ft_effnet.load_state_dict(state_dict)
for p in model_cb_ft_effnet.parameters():
        p.requires_grad = True
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_cb_ft_effnet.parameters()), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# setting up of Class Balanced Loss method
no_classes = 3
samples_per_class = torch.tensor([517, 163, 142]) # DR, Glaucoma, AMD - as stated in the project pdf (which I'll just blindly trust)
CB_loss = CB_Loss(samples_per_cls=samples_per_class, no_of_classes=no_classes, beta=0.9999, gamma=2.0)

C:\Users\netos\miniconda3\envs\llms\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


## Training and testing

In [31]:
model_cb_ft_effnet = train_model(model_cb_ft_effnet, train_csv, val_csv, test_csv, train_image_dir, val_image_dir, test_image_dir, scheduler=scheduler,
                                criterion=None, optimizer=optimizer, epochs=25, model_type="class_balance_fine_tuned", cb_loss=CB_loss)

Epoch 0/24
----------
[efficientnet] train: Epoch 1/25 Train Loss: 0.6401
[efficientnet] val: Epoch 1/25 Val Loss: 0.5659
Saved best model for efficientnet at checkpoints\best_efficientnet_class_balance_fine_tuned.pt with val loss: 0.5659163689613342
Epoch 1/24
----------
[efficientnet] train: Epoch 2/25 Train Loss: 0.3261
[efficientnet] val: Epoch 2/25 Val Loss: 0.4846
Saved best model for efficientnet at checkpoints\best_efficientnet_class_balance_fine_tuned.pt with val loss: 0.48459234952926633
Epoch 2/24
----------
[efficientnet] train: Epoch 3/25 Train Loss: 0.1970
[efficientnet] val: Epoch 3/25 Val Loss: 0.4560
Saved best model for efficientnet at checkpoints\best_efficientnet_class_balance_fine_tuned.pt with val loss: 0.45597842454910276
Epoch 3/24
----------
[efficientnet] train: Epoch 4/25 Train Loss: 0.1360
[efficientnet] val: Epoch 4/25 Val Loss: 0.4775
Epoch 4/24
----------
[efficientnet] train: Epoch 5/25 Train Loss: 0.1050
[efficientnet] val: Epoch 5/25 Val Loss: 0.4876
E

In [32]:
test_model(model_cb_ft_effnet, test_csv)

DR Results [efficientnet]
Accuracy : 0.7250
Precision: 0.6890
Recall   : 0.7131
F1-score : 0.6947
Kappa    : 0.3943
Glaucoma Results [efficientnet]
Accuracy : 0.8500
Precision: 0.8061
Recall   : 0.7697
F1-score : 0.7849
Kappa    : 0.5709
AMD Results [efficientnet]
Accuracy : 0.8900
Precision: 0.7241
Recall   : 0.7589
F1-score : 0.7396
Kappa    : 0.4797
----------
Average F1-score : 0.7396
